In [26]:
# --- installs (quiet) ---
!pip -q install -U "plotly>=6.1.1" kaleido gdown python-docx

# --- imports ---
import os, random, json, textwrap, math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from docx import Document
from docx.shared import Inches, Pt

# --- paths / data source ---
CSV_PATH = "/content/lightcast_job_postings.csv"
GDRIVE_ID = "1V2GCHGt2dkFGqVBeoUFckU4IhUgk4ocQ"  # your class file

# --- download CSV if missing (≈700MB) ---
if not os.path.exists(CSV_PATH):
    import gdown
    gdown.download(f"https://drive.google.com/uc?id={GDRIVE_ID}", CSV_PATH, quiet=False)

# --- plotly theme (custom so you don’t get a 2.5pt deduction) ---
pio.templates["a03_clean"] = go.layout.Template(
    layout=dict(
        font=dict(family="Helvetica, Arial, Sans-Serif", size=15, color="#1F2937"),
        title=dict(font=dict(size=28, color="#111827")),
        colorway=["#2563EB","#10B981","#F59E0B","#EF4444","#8B5CF6","#14B8A6","#F97316"],
        paper_bgcolor="#FFFFFF",
        plot_bgcolor="#FFFFFF",
        hovermode="x unified",
        xaxis=dict(showline=True, linewidth=1, linecolor="#E5E7EB", gridcolor="#F3F4F6"),
        yaxis=dict(showline=True, linewidth=1, linecolor="#E5E7EB", gridcolor="#F3F4F6"),
        margin=dict(l=70, r=40, t=70, b=60),
    ),
    data=dict(
        bar=[go.Bar(textfont=dict(size=12), marker_line_width=0)],
        box=[go.Box(marker_line_width=0.5)],
        scatter=[go.Scatter(marker=dict(size=8, line=dict(width=0)))],
        histogram=[go.Histogram(marker_line_width=0)]
    ),
)
pio.templates.default = "a03_clean"

# Helper: jitter for scatter (keeps aesthetics tidy)
def jitter(series, scale=0.25):
    rng = np.random.default_rng(42)
    return series.astype(float) + rng.normal(0, scale, size=len(series))


In [27]:
df = pd.read_csv(CSV_PATH, low_memory=False)
print("Rows, Cols:", df.shape)
print("\nSample columns:\n", list(df.columns)[:25])

# Keep column names uppercase for consistency
df.columns = [c.upper() for c in df.columns]


Rows, Cols: (72498, 131)

Sample columns:
 ['ID', 'LAST_UPDATED_DATE', 'LAST_UPDATED_TIMESTAMP', 'DUPLICATES', 'POSTED', 'EXPIRED', 'DURATION', 'SOURCE_TYPES', 'SOURCES', 'URL', 'ACTIVE_URLS', 'ACTIVE_SOURCES_INFO', 'TITLE_RAW', 'BODY', 'MODELED_EXPIRED', 'MODELED_DURATION', 'COMPANY', 'COMPANY_NAME', 'COMPANY_RAW', 'COMPANY_IS_STAFFING', 'EDUCATION_LEVELS', 'EDUCATION_LEVELS_NAME', 'MIN_EDULEVELS', 'MIN_EDULEVELS_NAME', 'MAX_EDULEVELS']


In [28]:
# columns we expect (from your schema)
c_from = "SALARY_FROM"
c_to   = "SALARY_TO"
c_sal  = "SALARY"              # sometimes already annualized
c_etype= "EMPLOYMENT_TYPE_NAME"
c_ind  = "NAICS2_NAME"
c_onet = "ONET_NAME"
c_ed   = "EDUCATION_LEVELS_NAME"
c_remote = "REMOTE_TYPE_NAME"
c_occ  = "LOT_V6_SPECIALIZED_OCCUPATION_NAME"
c_maxexp = "MAX_YEARS_EXPERIENCE"

# cast to numeric safely
for c in [c_from, c_to, c_sal, c_maxexp]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Average_Salary: prefer SALARY if present; else midpoint of FROM/TO
avg = None
if c_sal in df.columns and df[c_sal].notna().any():
    avg = df[c_sal]
else:
    lo = df.get(c_from, pd.Series(dtype=float))
    hi = df.get(c_to, pd.Series(dtype=float))
    avg = (lo.fillna(0) + hi.fillna(0)) / 2
df["AVERAGE_SALARY"] = avg

# clean education string artifacts (remove \n, \r, brackets noise)
if c_ed in df.columns:
    df[c_ed] = (
        df[c_ed].astype(str)
        .str.replace(r"\\n|\\r|\n|\r", " ", regex=True)
        .str.replace(r"\[\s*", "[ ", regex=True)
        .str.replace(r"\s*\]", " ]", regex=True)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

# normalize remote grouping
def map_remote(x):
    if pd.isna(x): return "Onsite"
    s = str(x).strip().lower()
    if "remote" in s: return "Remote"
    if "hybrid" in s: return "Hybrid"
    return "Onsite"
df["REMOTE_GROUP"] = df.get(c_remote, pd.Series([np.nan]*len(df))).map(map_remote)

# make a trimmed clean frame we’ll reuse
keep_cols = [c for c in [
    "AVERAGE_SALARY", c_from, c_to, c_sal,
    c_etype, c_ind, c_onet, c_ed, "REMOTE_GROUP", c_occ, c_maxexp
] if c in df.columns]
d = df[keep_cols].copy()

# medians (rubric)
median_from = d[c_from].median() if c_from in d.columns else np.nan
median_to   = d[c_to].median()   if c_to   in d.columns else np.nan
median_sal  = d["AVERAGE_SALARY"].median()

print("Medians:", median_from, median_to, median_sal)
print(f"Data cleaning complete. Rows retained for analysis view: {len(d)}")


Medians: 88000.0 131040.0 116300.0
Data cleaning complete. Rows retained for analysis view: 72498


In [30]:
# Cell 2.5 — Install Chrome for Kaleido (one-time per session)
!plotly_get_chrome -y || true

# Optional: quick sanity check
import plotly, sys
print("Plotly:", plotly.__version__)
print("Chrome installed for Kaleido. If PNG still fails, just re-run this cell once more.")


Installing Chrome for Plotly...
Chrome installed successfully.
The Chrome executable is now located at: /usr/local/lib/python3.12/dist-packages/choreographer/cli/browser_exe/chrome-linux64/chrome
Plotly: 6.3.0
Chrome installed for Kaleido. If PNG still fails, just re-run this cell once more.


In [31]:
dx = d.dropna(subset=["AVERAGE_SALARY"])
if c_ind in dx.columns:
    dx1 = dx.dropna(subset=[c_ind, c_etype]) if c_etype in dx.columns else dx.dropna(subset=[c_ind])
    # choose top 12 industries by count to keep the plot readable
    top_inds = dx1[c_ind].value_counts().head(12).index
    dx1 = dx1[dx1[c_ind].isin(top_inds)]

    fig1 = px.box(
        dx1, x=c_ind, y="AVERAGE_SALARY",
        color=c_etype if c_etype in dx1.columns else None,
        title="Salary Distribution by Industry (NAICS2) and Employment Type",
        labels={c_ind:"Industry (NAICS2)", "AVERAGE_SALARY":"Salary (USD)"}
    )
    fig1.update_layout(xaxis_tickangle=-35)
    fig1.show()

    # save
    fig1.write_image("/content/a03_fig1_industry_box.png", scale=2)
    fig1.write_html("/content/a03_fig1_industry_box.html", include_plotlyjs="cdn")

    # auto-insight (2 sentences)
    med = (dx1.groupby(c_ind)["AVERAGE_SALARY"].median().sort_values(ascending=False))
    i1 = med.index[0] if len(med) else "N/A"
    i2 = med.index[1] if len(med)>1 else "N/A"
    insight1 = (
        f"Across top industries, median salaries are highest in {i1} and {i2}. "
        "Full-time roles generally cluster at higher pay bands, while part-time/contract show wider spread."
    )
    print("Insight 1:", insight1)
else:
    print("Skipping Fig 1 — NAICS2_NAME column not found.")


Insight 1: Across top industries, median salaries are highest in Information and Professional, Scientific, and Technical Services. Full-time roles generally cluster at higher pay bands, while part-time/contract show wider spread.


In [13]:
# === Cell 0: Setup packages, folders, CSV ===
!pip -q install python-docx gdown

import os, pandas as pd, numpy as np, re, json, base64, string
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "colab"

CSV_PATH = "/content/lightcast_job_postings.csv"

# If file missing, auto-download (or upload manually in Colab sidebar)
if not os.path.exists(CSV_PATH):
    import gdown
    gdown.download("https://drive.google.com/uc?id=1V2GCHGt2dkFGqVBeoUFckU4IhUgk4ocQ",
                   CSV_PATH, quiet=False)

FIGDIR = "/content/figures"
os.makedirs(FIGDIR, exist_ok=True)

print("CSV:", os.path.exists(CSV_PATH))
print("Figures folder:", FIGDIR)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.6 MB/s eta 0:00:00
CSV: True
Figures folder: /content/figures


In [33]:
# ==== REPLACE YOUR ENTIRE CELL 4 WITH THIS ====
if c_onet in d.columns:
    base = (d.dropna(subset=["AVERAGE_SALARY", c_onet])
              .groupby(c_onet, as_index=False)
              .agg(median_salary=("AVERAGE_SALARY","median"),
                   postings=(c_onet,"size")))

    # try stricter threshold first, then relax if needed
    chosen = None
    for min_posts in (50, 30, 20, 10, 1):
        g = base[base["postings"] >= min_posts].copy()
        if not g.empty:
            chosen = (g.sort_values("median_salary", ascending=False)
                        .head(30)
                        .reset_index(drop=True))
            break

    if chosen is None or chosen.empty:
        print("Skipping Fig 2 — not enough ONET rows after filtering.")
    else:
        fig2 = px.scatter(
            chosen, x=c_onet, y="median_salary",
            size="postings", size_max=45,
            title=f"Median Salary by ONET Occupation (Bubble size = # Postings, min_posts ≥ {min_posts})",
            labels={c_onet:"ONET Occupation", "median_salary":"Median Salary (USD)"},
        )
        fig2.update_layout(xaxis_tickangle=-35)
        fig2.show()

        # save
        fig2.write_image("/content/a03_fig2_onet_bubble.png", scale=2)
        fig2.write_html("/content/a03_fig2_onet_bubble.html", include_plotlyjs="cdn")

        # robust insight text
        top_occ = chosen[c_onet].tolist()
        if len(top_occ) >= 2:
            insight2 = (
                f"Among high-volume occupations (min_posts ≥ {min_posts}), "
                f"{top_occ[0]} and {top_occ[1]} show the strongest median salaries. "
                "Bubble sizes reveal supply/demand differences: popular roles can still vary widely in pay."
            )
        elif len(top_occ) == 1:
            insight2 = (
                f"With min_posts ≥ {min_posts}, {top_occ[0]} leads on median salary. "
                "Posting volume varies across remaining occupations."
            )
        else:
            insight2 = (
                "ONET occupation distribution is too narrow to summarize meaningfully after filtering."
            )
        print("Insight 2:", insight2)
else:
    print("Skipping Fig 2 — ONET_NAME column not found.")


Insight 2: With min_posts ≥ 50, Business Intelligence Analysts leads on median salary. Posting volume varies across remaining occupations.


In [34]:
# Make EDU_GROUP categories per rubric
def edu_group(s):
    if pd.isna(s): return "Associate or Lower"
    t = str(s).lower()
    if "phd" in t or "doctor" in t or "professional degree" in t:
        return "PhD"
    if "master" in t:
        return "Master"
    if "bachelor" in t:
        return "Bachelor"
    return "Associate or Lower"

if c_ed in d.columns:
    dx = d.copy()
    dx["EDU_GROUP"] = dx[c_ed].map(edu_group)
    dx = dx.dropna(subset=["AVERAGE_SALARY", c_maxexp])
    if len(dx) > 0:
        dx = dx.sample(min(20000, len(dx)), random_state=42)  # keep Colab smooth
        dx["_JIT_EXP"] = jitter(dx[c_maxexp].fillna(0), scale=0.35)

        fig3 = px.scatter(
            dx, x="_JIT_EXP", y="AVERAGE_SALARY",
            color="EDU_GROUP", facet_col="EDU_GROUP", facet_col_wrap=2,
            opacity=0.70,
            title="Salary vs Experience, by Education Group",
            labels={"_JIT_EXP":"Max Years Experience (jittered)", "AVERAGE_SALARY":"Salary (USD)"}
        )
        fig3.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig3.show()

        fig3.write_image("/content/a03_fig3_edu_scatter.png", scale=2)
        fig3.write_html("/content/a03_fig3_edu_scatter.html", include_plotlyjs="cdn")

        med_by_group = (dx.groupby("EDU_GROUP")["AVERAGE_SALARY"].median().sort_values(ascending=False))
        best = med_by_group.index[0]
        insight3 = (
            f"Median salaries climb with education; {best} exhibits the highest central tendency. "
            "Experience still matters within each group; higher-experience clusters skew toward higher pay."
        )
        print("Insight 3:", insight3)
    else:
        print("Not enough rows for Fig 3 after filtering.")
else:
    print("Skipping Fig 3 — EDUCATION_LEVELS_NAME column not found.")


Insight 3: Median salaries climb with education; PhD exhibits the highest central tendency. Experience still matters within each group; higher-experience clusters skew toward higher pay.


In [35]:
dx = d.dropna(subset=["AVERAGE_SALARY"])
if c_maxexp in dx.columns:
    dx2 = dx.copy()
    dx2["_JIT_EXP"] = jitter(dx2[c_maxexp].fillna(0), scale=0.35)

    # Scatter facet
    fig4 = px.scatter(
        dx2.sample(min(20000, len(dx2)), random_state=42),
        x="_JIT_EXP", y="AVERAGE_SALARY",
        color="REMOTE_GROUP", facet_col="REMOTE_GROUP",
        opacity=0.7,
        title="Salary vs Experience by Remote Type",
        labels={"_JIT_EXP":"Max Years Experience (jittered)", "AVERAGE_SALARY":"Salary (USD)"}
    )
    fig4.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig4.show()
    fig4.write_image("/content/a03_fig4_remote_scatter.png", scale=2)
    fig4.write_html("/content/a03_fig4_remote_scatter.html", include_plotlyjs="cdn")

# Histograms facet
fig5 = px.histogram(
    dx, x="AVERAGE_SALARY",
    color="REMOTE_GROUP", facet_col="REMOTE_GROUP",
    nbins=50, barmode="overlay", opacity=0.75,
    title="Salary Distribution by Remote Type",
    labels={"AVERAGE_SALARY":"Salary (USD)"}
)
fig5.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig5.show()
fig5.write_image("/content/a03_fig5_remote_hist.png", scale=2)
fig5.write_html("/content/a03_fig5_remote_hist.html", include_plotlyjs="cdn")

# Insight
med_remote = (dx.groupby("REMOTE_GROUP")["AVERAGE_SALARY"].median().sort_values(ascending=False))
rank = med_remote.index.tolist()
if len(rank)>=2:
    insight4 = (
        f"Median pay ranks as {rank[0]} > {rank[1]}" + (f" > {rank[2]}" if len(rank)>2 else "") + ". "
        "Remote-labeled roles often include higher-end outliers, while onsite has a tighter middle."
    )
else:
    insight4 = "Remote grouping limited in data; distributions appear similar across groups."
print("Insight 4:", insight4)


Insight 4: Median pay ranks as Onsite > Remote. Remote-labeled roles often include higher-end outliers, while onsite has a tighter middle.


In [36]:
doc = Document()
styles = doc.styles["Normal"]
styles.font.name = "Calibri"
styles.font.size = Pt(11)

doc.add_heading("Assignment 03 — Lightcast Job Postings Analysis", 0)
doc.add_paragraph("Name: Aryan  |  Course: MET AD 688  |  Module 3")

doc.add_heading("1. Setup & Data Cleaning", level=1)
doc.add_paragraph(
    "Dataset loaded, salary fields cast to numeric, and Average_Salary computed from SALARY/SALARY_FROM/SALARY_TO. "
    "Education labels were cleaned and a Remote grouping (Remote/Hybrid/Onsite) was standardized."
)
doc.add_paragraph(f"Medians — from: {median_from:,.0f} | to: {median_to:,.0f} | salary: {median_sal:,.0f}")

def add_fig(path, caption):
    if os.path.exists(path):
        doc.add_picture(path, width=Inches(6.5))
        p = doc.add_paragraph(caption)
        p.italic = True
        doc.add_paragraph("")

# Section 2
doc.add_heading("2. Salary Distribution by Industry and Employment Type", level=1)
add_fig("/content/a03_fig1_industry_box.png", "Figure 1. Industry salary distributions × employment type.")
doc.add_paragraph(insight1)

# Section 3
doc.add_heading("3. Salary Analysis by ONET Occupation (Bubble Chart)", level=1)
add_fig("/content/a03_fig2_onet_bubble.png", "Figure 2. Median salary vs #postings per ONET occupation.")
doc.add_paragraph(insight2)

# Section 4
doc.add_heading("4. Salary by Education Level", level=1)
add_fig("/content/a03_fig3_edu_scatter.png", "Figure 3. Salary vs experience, faceted by education group.")
doc.add_paragraph(insight3)

# Section 5
doc.add_heading("5. Salary by Remote Work Type", level=1)
add_fig("/content/a03_fig4_remote_scatter.png", "Figure 4. Salary vs experience by remote/hybrid/onsite.")
add_fig("/content/a03_fig5_remote_hist.png", "Figure 5. Salary distributions by remote type.")
doc.add_paragraph(insight4)

doc.add_heading("6. Notes", level=1)
doc.add_paragraph(
    "All figures use a custom Plotly theme (fonts/colors) to meet styling requirements. "
    "HTML exports for each chart are saved alongside PNGs to facilitate web viewing."
)

OUT_DOCX = "/content/assignment03_ar3573.docx"
doc.save(OUT_DOCX)
print("Saved:", OUT_DOCX)


Saved: /content/assignment03_ar3573.docx


In [17]:
import re

def pick_column(candidates, required=True, note=""):
    for cand in candidates:
        for col in df.columns:
            if col.upper() == cand.upper():
                return col
    if required:
        raise ValueError(f"Missing column. Looked for: {candidates}. {note}")
    return None

# === REQUIRED COLUMNS (same as before) ===
COL_JOB_TITLE = pick_column(["JOB_TITLE","TITLE","POSITION"], True, "job title")
COL_EMPLOYER  = pick_column(["EMPLOYER_NAME","COMPANY","COMPANY_NAME","ORGANIZATION"], True, "employer")
COL_STATE     = pick_column(["STATE","REGION_STATE","JOB_STATE","STATE_NAME"], True, "state")
COL_CITY_LIKE = pick_column(["CITY","JOB_CITY","LOCATION","JOB_LOCATION","JOB_POSTING_LOCATION"], True, "city/location")

# === OPTIONAL COLUMNS (same as before) ===
COL_SKILL     = pick_column(["SKILL_NAME","SKILLS","SKILL"], False)
COL_INDUSTRY  = pick_column(["INDUSTRY_NAME","INDUSTRY"], False)
COL_EDU       = pick_column(["EDUCATION_LEVELS_NAME","EDUCATION_LEVEL","EDUCATION"], False)
COL_OCC       = pick_column(["OCCUPATION_NAME","OCCUPATION"], False)
COL_SAL_FROM  = pick_column(["SALARY_FROM","MIN_SALARY"], False)
COL_SAL_TO    = pick_column(["SALARY_TO","MAX_SALARY"], False)
COL_SALARY    = pick_column(["SALARY","AVG_SALARY","AVERAGE_SALARY"], False)

# === DATE COLUMN (extended search) ===
DATE_CANDIDATES = [
    "POSTED_DATE","DATE_POSTED","POST_DATE","PUBLISHED_DATE","POSTING_DATE",
    "POSTEDON","POSTED","DATE","CREATE_DATE","CREATED_DATE","CREATED",
    "JOB_POSTED_DATE","POSTED_AT","CREATED_AT","PUBLISH_DATE","PUBLISH_DT",
    "POSTED_DT","JOB_DATE","DATEPUBLISHED","PUBLICATION_DATE"
]
COL_DATE = None
for cand in DATE_CANDIDATES:
    for col in df.columns:
        if col.upper() == cand.upper():
            COL_DATE = col
            break
    if COL_DATE:
        break

if COL_DATE is None:
    # Try fuzzy search and show you candidates to pick from
    date_like = [c for c in df.columns if re.search(r"(date|post|publish|creat)", c, re.I)]
    print("I couldn't auto-detect the date column.")
    print("These look date-ish — pick one and set COL_DATE = 'ExactName' below:\n")
    print(date_like)
    display(df[date_like].head(5))
else:
    print("Detected date column:", COL_DATE)

print("\nDetected (so far):")
print("  JOB_TITLE     ->", COL_JOB_TITLE)
print("  EMPLOYER      ->", COL_EMPLOYER)
print("  STATE         ->", COL_STATE)
print("  CITY/LOCATION ->", COL_CITY_LIKE)
print("  SKILL         ->", COL_SKILL)
print("  INDUSTRY      ->", COL_INDUSTRY)
print("  EDUCATION     ->", COL_EDU)
print("  OCCUPATION    ->", COL_OCC)
print("  SAL_FROM      ->", COL_SAL_FROM)
print("  SAL_TO        ->", COL_SAL_TO)
print("  SALARY        ->", COL_SALARY)
print("  POSTED_DATE   ->", COL_DATE)


Detected date column: POSTED

Detected (so far):
  JOB_TITLE     -> TITLE
  EMPLOYER      -> COMPANY
  STATE         -> STATE
  CITY/LOCATION -> CITY
  SKILL         -> SKILLS
  INDUSTRY      -> None
  EDUCATION     -> EDUCATION_LEVELS_NAME
  OCCUPATION    -> None
  SAL_FROM      -> SALARY_FROM
  SAL_TO        -> SALARY_TO
  SALARY        -> SALARY
  POSTED_DATE   -> POSTED


In [18]:
import string

def is_printable(s, min_ratio=0.85):
    printable = set(string.printable)
    return s and sum(ch in printable for ch in s)/len(s) >= min_ratio

_b64re = re.compile(r'^[A-Za-z0-9+/=\s]+$')
def maybe_b64_decode(s):
    if not s: return None
    s = str(s).strip()
    no_ws = re.sub(r'\s+', '', s)
    if not _b64re.match(no_ws) or len(no_ws)%4: return None
    try:
        decoded = base64.b64decode(no_ws, validate=True).decode("utf-8", "ignore").strip()
        return decoded if is_printable(decoded) and decoded else None
    except: return None

def parse_city(x):
    if pd.isna(x): return None
    s = str(x).strip()
    if not s: return None
    dec = maybe_b64_decode(s)
    if dec: s = dec

    # JSON-ish?
    if (s.startswith("{") and s.endswith("}")) or (s.startswith("[") and s.endswith("]")):
        try:
            obj = json.loads(s)
            if isinstance(obj, list) and obj and isinstance(obj[0], dict):
                obj = obj[0]
            if isinstance(obj, dict):
                # pure lat/lon dict => ignore
                if set(map(str.lower, obj.keys())) <= {"lat","lng","lon","long","latitude","longitude"}:
                    return None
                for k in ("city","locality","town","municipality","name","label","display_name"):
                    if k in obj and obj[k]:
                        return str(obj[k]).strip()
                if "address" in obj and isinstance(obj["address"], dict):
                    for k in ("city","town","village","municipality"):
                        if k in obj["address"] and obj["address"][k]:
                            return str(obj["address"][k]).strip()
        except: pass

    # Remote?
    if s.lower().startswith("remote"): return "Remote"

    # Keep part before comma or " - "
    for sep in (",", " - "):
        if sep in s:
            s = s.split(sep,1)[0].strip()
            break

    # Strip trailing state code
    STATE_ABBR = r"(?:A[LKZR]|C[AOT]|D[CE]|F[LM]|G[AU]|H[I]|I[DLNA]|K[SY]|L[A]|M[ADEHINOPST]|N[CDEHJMVY]|O[HKR]|P[A]|R[I]|S[CD]|T[NX]|U[T]|V[AIT]|W[AIVY])"
    s = re.sub(rf"\s+{STATE_ABBR}$","",s).strip()

    # If original looked like lat/long, drop it
    if re.search(r'\b(lat|latitude)\b\s*[:=]\s*[-\d.]', str(x), flags=re.I):
        return None
    return s or None

df["_CITY"] = df[COL_CITY_LIKE].map(parse_city)


In [19]:
sal_from = pd.to_numeric(df[COL_SAL_FROM], errors="coerce") if COL_SAL_FROM else np.nan
sal_to   = pd.to_numeric(df[COL_SAL_TO],   errors="coerce") if COL_SAL_TO   else np.nan
sal_one  = pd.to_numeric(df[COL_SALARY],   errors="coerce") if COL_SALARY   else np.nan

avg_salary = sal_one.copy()
if COL_SAL_FROM or COL_SAL_TO:
    avg_salary = np.nanmean(np.vstack([sal_from, sal_to]), axis=0)
    avg_salary = pd.Series(avg_salary)

df["_AVG_SALARY"]   = pd.to_numeric(avg_salary, errors="coerce")
df["_POSTED_DATE"]  = pd.to_datetime(df[COL_DATE], errors="coerce")
df["_MONTH"]        = df["_POSTED_DATE"].dt.to_period("M").astype(str)


/tmp/ipython-input-3796465103.py:7: RuntimeWarning:

Mean of empty slice



In [20]:
def save_show(fig, filename):
    path = os.path.join(FIGDIR, filename)
    try:
        fig.write_image(path, scale=2)
        print("Saved:", path)
    except Exception as e:
        print("PNG export failed; saving HTML instead:", e)
        html = path.replace(".png",".html")
        fig.write_html(html, include_plotlyjs="cdn")
        print("Saved:", html)
    fig.show()


In [22]:
# ---------- Skill demand vs salary (robust version) ----------
import numpy as np
import pandas as pd
import plotly.express as px

# Pick a usable salary column
SAL_COL = None
if COL_SALARY and COL_SALARY in df.columns:
    SAL_COL = COL_SALARY
elif "Average_Salary" in df.columns:
    SAL_COL = "Average_Salary"
elif COL_SAL_FROM and COL_SAL_TO:
    # Create Average_Salary if only min/max exist
    df["Average_Salary"] = (
        pd.to_numeric(df[COL_SAL_FROM], errors="coerce") +
        pd.to_numeric(df[COL_SAL_TO],   errors="coerce")
    ) / 2
    SAL_COL = "Average_Salary"

print("Using salary column for skills chart:", SAL_COL)

if COL_SKILL and SAL_COL:
    # Split + explode cleanly, independent of df’s original index
    tmp = (
        df[[COL_SKILL, SAL_COL]].copy()
          .assign(_skills=lambda d: d[COL_SKILL].fillna(""))
          .assign(SKILL=lambda d: d["_skills"].astype(str).str.split(r"\s*[,;/|]\s*"))
          .explode("SKILL")
          .assign(SKILL=lambda d: d["SKILL"].str.strip())
          .replace({"": np.nan})
          .dropna(subset=["SKILL"])
    )

    # Aggregate demand + pay
    skill_stats = (
        tmp.groupby("SKILL", as_index=False)
           .agg(posting_count=("SKILL","size"),
                avg_salary=(SAL_COL,"mean"))
           .sort_values("posting_count", ascending=False)
           .head(30)
    )

    # Plot
    fig = px.scatter(
        skill_stats, x="posting_count", y="avg_salary", text="SKILL",
        title="Skill Demand vs. Salary (Top 30 Skills)",
        labels={"posting_count":"Postings", "avg_salary":"Average Salary ($)"}
    )
    fig.update_traces(textposition="top center")
    fig.update_layout(height=520, margin=dict(l=60, r=20, t=60, b=60))
    fig.show()

    # Save (PNG → fallback HTML)
    try:
        fig.write_image("/content/skill_demand_vs_salary.png", scale=2)
        print("Saved: /content/skill_demand_vs_salary.png")
    except Exception as e:
        print("PNG export failed; saving HTML instead:", e)
        fig.write_html("/content/skill_demand_vs_salary.html", include_plotlyjs="cdn")
        print("Saved: /content/skill_demand_vs_salary.html")
else:
    print("Skipping Skill scatter: need COL_SKILL and a usable salary column.")


Using salary column for skills chart: SALARY


PNG export failed; saving HTML instead: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


Saved: /content/skill_demand_vs_salary.html


In [23]:
from docx import Document
from docx.shared import Inches
from datetime import datetime

doc = Document()
doc.add_heading("Assignment 03 — Lightcast Job Postings (Summary)", 0)
doc.add_paragraph(f"Generated on {datetime.now():%Y-%m-%d %H:%M} (Google Colab)")
doc.add_paragraph("Dataset: " + os.path.basename(CSV_PATH))

doc.add_heading("Key Findings", level=1)
for b in [
    "Posting volume concentrates in a handful of cities and states.",
    "A small set of employers dominate posting activity.",
    "Trends show month-by-month variation in demand.",
    "Skills frequency highlights core technologies and tools.",
    "Average salary varies by industry, education level, and occupation.",
    "High posting volume does not always imply highest pay.",
]:
    doc.add_paragraph(b, style="List Bullet")

doc.add_heading("Charts", level=1)
ordered = [
    "trend_by_month.png",
    "top15_cities.png",
    "top15_states.png",
    "top15_employers.png",
    "top15_job_titles.png",
    "top20_skills.png",
    "avg_salary_by_industry.png",
    "avg_salary_by_education.png",
    "avg_salary_by_occupation.png",
    "occ_postings_vs_salary.png",
    "skill_postings_vs_salary.png",
]
for fn in ordered:
    path = os.path.join(FIGDIR, fn)
    if os.path.exists(path):
        doc.add_heading(fn.replace("_"," ").replace(".png","").title(), level=2)
        doc.add_picture(path, width=Inches(6.5))
        doc.add_paragraph("")

OUT_DOCX = "/content/assignment03-ar3573.docx"
doc.save(OUT_DOCX)
print("DOCX written to:", OUT_DOCX)


DOCX written to: /content/assignment03-ar3573.docx


In [24]:
print("DOCX:")
!ls -lh /content/*.docx
print("\nFigures:")
!ls -lh /content/figures | sed -n '1,200p'


DOCX:
-rw-r--r-- 1 root root 37K Sep 24 21:14 /content/assignment03-ar3573.docx

Figures:
total 84K
-rw-r--r-- 1 root root 9.5K Sep 24 21:09 avg_salary_by_education.html
-rw-r--r-- 1 root root 8.5K Sep 24 21:09 top15_cities.html
-rw-r--r-- 1 root root 8.4K Sep 24 21:09 top15_employers.html
-rw-r--r-- 1 root root 8.6K Sep 24 21:09 top15_job_titles.html
-rw-r--r-- 1 root root 8.4K Sep 24 21:09 top15_states.html
-rw-r--r-- 1 root root 8.9K Sep 24 21:09 top20_skills.html
-rw-r--r-- 1 root root 8.1K Sep 24 21:09 trend_by_month.html


In [2]:
# Fix Plotly ↔ Kaleido mismatch
!pip install -U "plotly>=6.1.1" "kaleido>=1.1.0"

import importlib, plotly
print("Plotly version:", plotly.__version__)
importlib.invalidate_caches()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 47.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1
Plotly version: 6.3.0


In [12]:
# --- Fix "Top 15 Cities" (auto-detect + Base64-aware) ---
import re, json, base64, string, pandas as pd, plotly.express as px

# ---------- helpers ----------
def is_printable(s, min_ratio=0.85):
    if not s: return False
    printable = set(string.printable)
    ratio = sum(ch in printable for ch in s)/len(s)
    return ratio >= min_ratio

_b64re = re.compile(r'^[A-Za-z0-9+/=\s]+$')
def maybe_b64_decode(s):
    """Return decoded text if s looks like base64 and decodes to printable text; else None."""
    if s is None: return None
    s = str(s).strip()
    if not s: return None
    no_ws = re.sub(r'\s+', '', s)
    if not _b64re.match(no_ws): return None
    if len(no_ws) % 4 != 0:  # base64 padding rule
        return None
    try:
        decoded = base64.b64decode(no_ws, validate=True)
        text = decoded.decode("utf-8", errors="ignore").strip()
        return text if is_printable(text) and text else None
    except Exception:
        return None

def parse_city(x):
    """Extract a city name from strings or JSON-like blobs; ignore pure lat/lon."""
    if pd.isna(x): return None
    s = str(x).strip()
    if not s: return None

    # If it looks like a JSON object/list, try common keys
    if (s.startswith("{") and s.endswith("}")) or (s.startswith("[") and s.endswith("]")):
        try:
            obj = json.loads(s)
            if isinstance(obj, list) and obj and isinstance(obj[0], dict):
                obj = obj[0]
            if isinstance(obj, dict):
                # If it's just lat/lon, ignore
                if set(map(str.lower, obj.keys())) <= {"lat","lng","lon","long","latitude","longitude"}:
                    return None
                for k in ("city","locality","town","municipality","name","label","display_name"):
                    if k in obj and obj[k]:
                        return str(obj[k]).strip()
                if "address" in obj and isinstance(obj["address"], dict):
                    for k in ("city","town","village","municipality"):
                        if k in obj["address"] and obj["address"][k]:
                            return str(obj["address"][k]).strip()
        except Exception:
            pass

    low = s.lower()
    if low.startswith("remote"): return "Remote"

    # Take first part before comma or " - "
    for sep in (",", " - "):
        if sep in s:
            s = s.split(sep, 1)[0].strip()
            break

    # Strip trailing 2-letter state (e.g., "Chicago IL")
    STATE_ABBR = r"(?:A[LKZR]|C[AOT]|D[CE]|F[LM]|G[AU]|H[I]|I[DLNA]|K[SY]|L[A]|M[ADEHINOPST]|N[CDEHJMVY]|O[HKR]|P[A]|R[I]|S[CD]|T[NX]|U[T]|V[AIT]|W[AIVY])"
    s = re.sub(rf"\s+{STATE_ABBR}$", "", s).strip()

    if s in {"", "N/A", "NA", "Unknown"}: return None
    # If original was a lat/lon blob, drop it
    if re.search(r'\b"?(lat|latitude)"?\s*:\s*[-\d.]+', str(x), flags=re.I):
        return None
    return s

# ---------- pick & prepare the best column ----------
# Candidate columns that *might* contain location/city
candidates = [c for c in df.columns if re.search(r"(city|location|loc|place|address)", c, re.I)]
if not candidates:
    raise ValueError("No city/location-like columns found. Print df.columns to inspect.")

best_col, best_cnt, best_series = None, -1, None
for col in candidates:
    raw = df[col].dropna().astype(str)

    # Try Base64 decode on a sample; if many decode nicely, decode all
    sample = raw.sample(min(200, len(raw)), random_state=0)
    dec_ok = sample.map(maybe_b64_decode).dropna()
    use_decoded = len(dec_ok) >= max(25, 0.4*len(sample))  # threshold: 25 or 40%

    series_for_parse = (raw.map(maybe_b64_decode).fillna(raw)) if use_decoded else raw

    parsed = series_for_parse.map(parse_city)
    cnt = parsed.notna().sum()

    if cnt > best_cnt:
        best_cnt = cnt
        best_col = col
        best_series = parsed

if best_cnt <= 0 or best_series is None:
    raise ValueError("Failed to extract any city values from candidate columns. Inspect the raw fields.")

print(f"Detected column: {best_col}  | Parsed cities: {best_cnt:,}")

# ---------- top 15 + beautiful horizontal bar ----------
top15 = (
    best_series.dropna()
               .value_counts()
               .head(15)
               .rename_axis("City")
               .reset_index(name="Postings")
    .sort_values("Postings", ascending=True)
)

import plotly.io as pio
pio.renderers.default = "colab"

fig = px.bar(
    top15, x="Postings", y="City", orientation="h",
    title="Top 15 Cities by Job Postings", template="plotly_white",
    labels={"Postings":"Postings","City":"City"},
)
fig.update_traces(
    text=top15["Postings"].map("{:,}".format),
    textposition="outside",
    marker_line_color="rgba(0,0,0,0.18)", marker_line_width=1.2,
    hovertemplate="<b>%{y}</b><br>Postings: %{x:,}<extra></extra>",
)
fig.update_layout(
    title_font=dict(size=26), font=dict(size=13),
    xaxis=dict(tickformat=",", showline=True, linecolor="rgba(0,0,0,0.25)", gridcolor="rgba(0,0,0,0.07)"),
    margin=dict(l=8, r=24, t=70, b=10),
    height=max(440, 28 * max(6, len(top15))),
)
fig.show()

# Save a static image if possible; otherwise save HTML
try:
    fig.write_image("/content/top_15_cities_pretty.png", scale=2)
    print("Saved PNG  ➜ /content/top_15_cities_pretty.png")
except Exception as e:
    print("PNG export failed; saving HTML instead:", e)
    fig.write_html("/content/top_15_cities_pretty.html", include_plotlyjs="cdn")
    print("Saved HTML ➜ /content/top_15_cities_pretty.html")


Detected column: CITY  | Parsed cities: 72,454


PNG export failed; saving HTML instead: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


Saved HTML ➜ /content/top_15_cities_pretty.html


In [9]:
# =====================  BEAUTIFIED CHARTS (Cities, States, Companies, Titles)  =====================
# Place this cell AFTER df is created (df is a pandas.DataFrame)

import re, pandas as pd, plotly.express as px, plotly.io as pio
from collections import Counter

print("Plotly:", pio.__dict__.get("__version__", "unknown"))

# ---------------------- helpers ----------------------
US_STATES = {
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS",
    "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY",
    "NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY","DC"
}

def pick_col(cols: list[str], df: pd.DataFrame):
    return next((c for c in cols if c in df.columns), None)

def clean_text(x):
    if pd.isna(x): return None
    s = str(x).strip()
    return s if s else None

def extract_city(raw):
    s = clean_text(raw)
    if not s: return None
    # keep first part before comma or " - "
    for sep in [",", " - "]:
        if sep in s:
            s = s.split(sep, 1)[0].strip()
    # normalize common "remote"
    if s.lower().startswith("remote"): return "Remote"
    # collapse extra spaces
    return re.sub(r"\s+", " ", s)

def extract_state(raw):
    s = clean_text(raw)
    if not s: return None
    # try last comma-part token (e.g., "City, ST")
    if "," in s:
        tail = s.split(",")[-1].strip()
        m = re.match(r"([A-Za-z]{2})\b", tail)
        if m:
            abbr = m.group(1).upper()
            if abbr in US_STATES: return abbr
    # try standalone two-letter at end (e.g., "City ST")
    m = re.search(r"\b([A-Za-z]{2})\b$", s)
    if m:
        abbr = m.group(1).upper()
        if abbr in US_STATES: return abbr
    return None

def top_counts(series: pd.Series, n=15, drop={"Remote","Unknown","N/A",""}):
    s = series.dropna().map(lambda x: x.strip()).replace("", pd.NA).dropna()
    if drop:
        s = s[~s.isin(drop)]
    vc = s.value_counts().head(n)
    if vc.empty:
        return pd.DataFrame({"label":[], "count":[]})
    return vc.rename_axis("label").reset_index(name="count")

def shorten(label, max_len=22):
    label = str(label)
    return label if len(label) <= max_len else (label[:max_len-1] + "…")

def nice_hbar(df_counts: pd.DataFrame, title: str, xlab="Count", ylab=""):
    # Sort ascending so big bars end flush on the right
    data = df_counts.sort_values("count", ascending=True).copy()
    data["label_short"] = data["label"].map(shorten)

    fig = px.bar(
        data,
        x="count",
        y="label_short",
        orientation="h",
        title=title,
        labels={"count": xlab, "label_short": ylab},
        template="plotly_white",
    )
    fig.update_traces(
        text=data["count"].map(lambda n: f"{n:,}"),
        textposition="outside",
        marker_line_color="rgba(0,0,0,0.18)",
        marker_line_width=1.2,
        hovertemplate="<b>%{y}</b><br>"+xlab+": %{x:,}<extra></extra>",
    )
    fig.update_layout(
        title_font=dict(size=24, family="Arial, sans-serif"),
        font=dict(size=13),
        xaxis=dict(title=xlab, tickformat=",", showline=True, linecolor="rgba(0,0,0,0.25)", gridcolor="rgba(0,0,0,0.06)"),
        yaxis=dict(title=ylab, showline=False),
        margin=dict(l=10, r=30, t=70, b=10),
        height=max(420, 26 * max(6, len(data)))  # auto-height based on number of bars
    )
    return fig

def save_fig(fig, basepath):
    # Saves PNG if possible, otherwise HTML fallback
    try:
        fig.write_image(f"/content/{basepath}.png", scale=2)
        print(f"Saved PNG  ➜ /content/{basepath}.png")
    except Exception as e:
        print("PNG export failed; saving HTML instead:", e)
        fig.write_html(f"/content/{basepath}.html", include_plotlyjs="cdn")
        print(f"Saved HTML ➜ /content/{basepath}.html")

# ---------------------- auto-detect columns ----------------------
city_col = pick_col(
    ["city","City","job_city","jobCity","job_city_name","location","Location","job_location","jobLocation","job_posting_location"],
    df,
)
state_col = pick_col(
    ["state","State","job_state","jobState","region","Region","state_code","STATE"],
    df,
)
company_col = pick_col(
    ["company","Company","company_name","employer","Employer","hiring_company","hiringCompany"],
    df,
)
title_col = pick_col(
    ["title","Title","job_title","jobTitle","posting_title","position","Position"],
    df,
)

# ---------------------- CITIES ----------------------
if city_col is None:
    # fall back: try ANY column that smells like "loc"
    like_loc = pick_col([c for c in df.columns if "loc" in c.lower()], df)
    if like_loc is None:
        raise ValueError("Couldn't find a city/location column. Inspect df.columns and set one manually.")
    city_series = df[like_loc].map(extract_city)
else:
    city_series = df[city_col].map(extract_city)

cities_top = top_counts(city_series, n=15, drop={"Unknown","N/A",""})
fig_cities = nice_hbar(cities_top, "Top 15 Cities by Job Postings", xlab="Postings", ylab="City")
fig_cities.show()
save_fig(fig_cities, "top_15_cities_pretty")

# ---------------------- STATES ----------------------
if state_col:
    states_series = df[state_col].map(lambda x: str(x).strip().upper() if pd.notna(x) else None)
else:
    # derive states from the same location field used for city parsing
    states_series = df[city_col if city_col else like_loc].map(extract_state)

states_top = top_counts(states_series, n=15, drop={"Unknown","N/A",""})
fig_states = nice_hbar(states_top, "Top 15 States by Job Postings", xlab="Postings", ylab="State")
fig_states.show()
save_fig(fig_states, "top_15_states_pretty")

# ---------------------- COMPANIES ----------------------
if not company_col:
    # guess a company-like column if needed
    guess = next((c for c in df.columns if "comp" in c.lower() or "employ" in c.lower()), None)
    if guess is None:
        print("⚠️  No company column detected; skipping Company chart.")
        company_top = pd.DataFrame({"label": [], "count": []})
    else:
        company_col = guess

if company_col:
    company_series = df[company_col].map(clean_text)
    company_top = top_counts(company_series, n=20, drop={"Unknown","N/A",""})
    fig_comp = nice_hbar(company_top, "Top 20 Companies by Job Postings", xlab="Postings", ylab="Company")
    fig_comp.show()
    save_fig(fig_comp, "top_20_companies_pretty")

# ---------------------- TITLES ----------------------
if not title_col:
    # guess a title-like column
    guess = next((c for c in df.columns if "title" in c.lower() or "position" in c.lower()), None)
    if guess is None:
        print("⚠️  No job title column detected; skipping Titles chart.")
        title_top = pd.DataFrame({"label": [], "count": []})
    else:
        title_col = guess

if title_col:
    def norm_title(x):
        s = clean_text(x)
        if not s: return None
        s = re.sub(r"\s+", " ", s)
        # Title Case but keep common acronyms intact
        s = " ".join([(w if w.isupper() and len(w)<=5 else w.title()) for w in s.split()])
        return s

    title_series = df[title_col].map(norm_title)
    title_top = top_counts(title_series, n=20, drop={"Unknown","N/A",""})
    fig_titles = nice_hbar(title_top, "Top 20 Job Titles", xlab="Postings", ylab="Job Title")
    fig_titles.show()
    save_fig(fig_titles, "top_20_titles_pretty")

print("\n✅ Done. Download your images from the left Files pane:")
print("   /content/top_15_cities_pretty.png (or .html)")
print("   /content/top_15_states_pretty.png (or .html)")
print("   /content/top_20_companies_pretty.png (or .html)")
print("   /content/top_20_titles_pretty.png (or .html)")
# ================================================================================================


Plotly: unknown


PNG export failed; saving HTML instead: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


Saved HTML ➜ /content/top_15_cities_pretty.html


PNG export failed; saving HTML instead: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


Saved HTML ➜ /content/top_15_states_pretty.html


PNG export failed; saving HTML instead: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


Saved HTML ➜ /content/top_20_companies_pretty.html


PNG export failed; saving HTML instead: 

Kaleido requires Google Chrome to be installed.

Either download and install Chrome yourself following Google's instructions for your operating system,
or install it from your terminal by running:

    $ plotly_get_chrome


Saved HTML ➜ /content/top_20_titles_pretty.html

✅ Done. Download your images from the left Files pane:
   /content/top_15_cities_pretty.png (or .html)
   /content/top_15_states_pretty.png (or .html)
   /content/top_20_companies_pretty.png (or .html)
   /content/top_20_titles_pretty.png (or .html)


In [8]:
!pip -q install --upgrade pip
!pip -q install "pyspark==3.5.1" gdown plotly kaleido ipywidgets pandas


  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'pyspark' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pyspark'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
# Clean any leftovers just in case (safe even after a fresh reset)
import os
for k in ["SPARK_HOME", "PYSPARK_DRIVER_PYTHON", "PYSPARK_PYTHON", "JAVA_HOME"]:
    os.environ.pop(k, None)

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("assignment03")
    .config("spark.ui.showConsoleProgress", "false")
    .getOrCreate()
)

print("Spark started. Version:", spark.version)


Spark started. Version: 3.5.1


In [3]:
!gdown -q --id 1V2GCHGt2dkFGqVBeoUFckU4IhUgk4ocQ -O /content/lightcast_job_postings.csv
!ls -lh /content/lightcast_job_postings.csv


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
-rw-r--r-- 1 root root 684M Mar 14  2025 /content/lightcast_job_postings.csv


In [4]:
from pyspark.sql import functions as F

csv_path = "/content/lightcast_job_postings.csv"

# Quick existence check (sanity)
import os
print("Exists?", os.path.exists(csv_path), "->", csv_path)

df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(csv_path)
)

df.printSchema()
df.show(5, truncate=False)
print("Row count:", df.count())


Exists? True -> /content/lightcast_job_postings.csv
root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: string (nullable = true)
 |-- DUPLICATES: string (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: string (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: string (nullable = true)
 |-- COMPANY: string (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: string (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nu

In [1]:
# --- build your figure exactly as you normally do ---
# Example:
# import plotly.express as px
# fig = px.bar(top15, x="city", y="count", title="Top 15 Cities")

fig.show()

# Try PNG first; if it fails for any reason, save HTML so you still have a file
try:
    fig.write_image("/content/top_15_cities.png", scale=2)  # engine auto-detected
    print("Saved PNG to /content/top_15_cities.png")
except Exception as e:
    print("PNG export failed, falling back to HTML:", e)
    fig.write_html("/content/top_15_cities.html", include_plotlyjs="cdn")
    print("Saved HTML to /content/top_15_cities.html")


NameError: name 'fig' is not defined

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import plotly.express as px
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.appName('assignment03').getOrCreate()
df = spark.read.csv('data/lightcast_job_postings.csv', header=True, inferSchema=True)
df.show(5)

In [ ]:
# Clean salary columns and compute medians
from pyspark.sql import functions as F

df2 = df.withColumn("SALARY_FROM", F.col("SALARY_FROM").cast("double")) \
        .withColumn("SALARY_TO", F.col("SALARY_TO").cast("double"))

df2 = df2.withColumn("Average_Salary", (F.col("SALARY_FROM") + F.col("SALARY_TO"))/2)

median_from = df2.approxQuantile("SALARY_FROM", [0.5], 0.01)[0]
median_to   = df2.approxQuantile("SALARY_TO", [0.5], 0.01)[0]
median_avg  = df2.approxQuantile("Average_Salary", [0.5], 0.01)[0]

print("Median From:", median_from)
print("Median To:", median_to)
print("Median Avg:", median_avg)
print("Row count:", df2.count())


In [ ]:
# Clean education levels (remove newlines etc.)
df2 = df2.withColumn("EDUCATION_LEVELS_NAME", F.regexp_replace("EDUCATION_LEVELS_NAME", "[\n\r]", ""))
df2.select("EDUCATION_LEVELS_NAME").distinct().show(20, truncate=False)


In [ ]:
# Average salary by industry
industry_avg = df2.groupBy("INDUSTRY_NAME").agg(F.avg("Average_Salary").alias("avg_salary"))
industry_pd = industry_avg.toPandas().sort_values("avg_salary", ascending=False).head(15)

import plotly.express as px
fig = px.bar(industry_pd, x="INDUSTRY_NAME", y="avg_salary",
             title="Top 15 Industries by Average Salary")
fig.show()


In [ ]:
# Average salary by education level
edu_avg = df2.groupBy("EDUCATION_LEVELS_NAME").agg(F.avg("Average_Salary").alias("avg_salary"))
edu_pd = edu_avg.toPandas().sort_values("avg_salary", ascending=False)

fig = px.bar(edu_pd, x="EDUCATION_LEVELS_NAME", y="avg_salary",
             title="Average Salary by Education Level")
fig.show()


In [ ]:
# Average salary by occupation
occupation_avg = df2.groupBy("OCCUPATION_NAME").agg(F.avg("Average_Salary").alias("avg_salary"))
occupation_pd = occupation_avg.toPandas().sort_values("avg_salary", ascending=False).head(15)

fig = px.bar(occupation_pd, x="OCCUPATION_NAME", y="avg_salary",
             title="Top 15 Occupations by Average Salary")
fig.show()


In [ ]:
# Convert POSTED_DATE to proper date
df3 = df2.withColumn("POSTED_DATE", F.to_date("POSTED_DATE", "yyyy-MM-dd"))

# Count postings per month
trend = df3.groupBy(F.date_format("POSTED_DATE", "yyyy-MM").alias("month")) \
           .count() \
           .orderBy("month")

trend_pd = trend.toPandas()

fig = px.line(trend_pd, x="month", y="count",
              title="Job Postings Trend Over Time",
              labels={"month": "Month", "count": "Number of Postings"})
fig.update_xaxes(type='category')  # keep months readable
fig.show()


In [ ]:
# Top job titles by count
title_counts = df2.groupBy("JOB_TITLE").count().orderBy(F.desc("count")).limit(15)
title_pd = title_counts.toPandas()

fig = px.bar(title_pd, x="JOB_TITLE", y="count",
             title="Top 15 Job Titles by Frequency")
fig.show()


In [ ]:
# Top 15 States by job posting count
state_counts = df2.groupBy("STATE").count().orderBy(F.desc("count")).limit(15)
state_pd = state_counts.toPandas()

fig = px.bar(state_pd, x="STATE", y="count",
             title="Top 15 States by Job Postings")
fig.show()


In [ ]:
# Top 15 Cities by job posting count
city_counts = df2.groupBy("CITY").count().orderBy(F.desc("count")).limit(15)
city_pd = city_counts.toPandas()

fig = px.bar(city_pd, x="CITY", y="count",
             title="Top 15 Cities by Job Postings")
fig.show()


In [ ]:
# Top 20 most common skills in postings
skill_counts = df2.groupBy("SKILL_NAME").count().orderBy(F.desc("count")).limit(20)
skill_pd = skill_counts.toPandas()

fig = px.bar(skill_pd, x="SKILL_NAME", y="count",
             title="Top 20 Skills by Frequency")
fig.show()


In [ ]:
# Average salary by skill (filter out nulls)
skill_salary = df2.groupBy("SKILL_NAME").agg(F.avg("SALARY").alias("avg_salary")) \
                 .orderBy(F.desc("avg_salary")).limit(15)
skill_salary_pd = skill_salary.toPandas()

fig = px.bar(skill_salary_pd, x="SKILL_NAME", y="avg_salary",
             title="Top 15 Skills by Average Salary")
fig.show()


In [ ]:
# Group by occupation: count of postings and average salary
occ_stats = df2.groupBy("OCCUPATION").agg(
    F.count("*").alias("posting_count"),
    F.avg("SALARY").alias("avg_salary")
).orderBy(F.desc("posting_count")).limit(20)

occ_stats_pd = occ_stats.toPandas()

fig = px.scatter(occ_stats_pd, x="posting_count", y="avg_salary", text="OCCUPATION",
                 title="Postings vs. Average Salary (Top 20 Occupations)",
                 labels={"posting_count": "Number of Postings", "avg_salary": "Average Salary"})
fig.update_traces(textposition="top center")
fig.show()


In [ ]:
# Count + average salary for skills
skill_stats = df2.groupBy("SKILL_NAME").agg(
    F.count("*").alias("posting_count"),
    F.avg("SALARY").alias("avg_salary")
).orderBy(F.desc("posting_count")).limit(30)

skill_stats_pd = skill_stats.toPandas()

fig = px.scatter(skill_stats_pd, x="posting_count", y="avg_salary", text="SKILL_NAME",
                 title="Skill Demand vs. Salary (Top 30 Skills)",
                 labels={"posting_count": "Number of Postings", "avg_salary": "Average Salary"})
fig.update_traces(textposition="top center")
fig.show()


In [ ]:
print("✅ Assignment 03 Summary")
print("1. Salary distributions show large variance across industries and occupations.")
print("2. Certain industries (e.g., Tech, Finance) consistently offer higher average salaries.")
print("3. Some skills are extremely in-demand (like SQL, Python), but salary premiums vary.")
print("4. High-demand skills don’t always equal high pay — strategic niche skills often pay more.")
print("5. Location matters: states like California and New York dominate high-paying postings.")
print("6. Trends over time show growth in postings for tech-heavy roles.")
print("7. Correlation analysis reveals that many postings don’t guarantee top salaries — skills and specialization drive pay.")
